<a href="https://colab.research.google.com/github/Kirutkar/airesidency/blob/main/pineconeindexdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/PineconeIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone Vector Store

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index llama-index-vector-stores-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Creating a Pinecone Index

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Google Colab secrets
openai.api_key = userdata.get('openai')

if openai.api_key:
    os.environ["OPENAI_API_KEY"] = openai.api_key

In [5]:
#PUT Tyour aopi key

from google.colab import userdata


api_key = userdata.get('PINECONE_API_KEY')

if api_key:
    os.environ["PINECONE_API_KEY"] = api_key


pc = Pinecone(api_key=api_key)

In [6]:
#delete if needed
pc.delete_index("quickstart")

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '4e293ec8b9df1d451a1215db219f8b17', 'date': 'Wed, 18 Jun 2025 18:51:13 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource quickstart not found"},"status":404}


In [7]:
# dimensions are for text-embedding-ada-002

pc.create_index(
    name="quickstart",
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


{
    "name": "quickstart",
    "metric": "euclidean",
    "host": "quickstart-o2k0uw8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:
pinecone_index = pc.Index("quickstart")

#### Load documents, build the PineconeVectorStore and VectorStoreIndex

In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display

Download Data

In [12]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-06-18 18:57:21--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2025-06-18 18:57:21 (6.23 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [13]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [14]:
documents

[Document(id_='4a14e63e-a217-4698-859c-278e70e71938', embedding=None, metadata={'file_path': '/content/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2025-06-18', 'last_modified_date': '2025-06-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly an

In [15]:
# initialize without metadata filter
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Upserted vectors:   0%|          | 0/22 [00:00<?, ?it/s]

#### Query Index

May take a minute or so for the index to be ready!

In [16]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("what is this document about")

In [17]:
display(Markdown(f"<b>{response}</b>"))

<b>The document is about the author's reflections on his experiences with writing essays, choosing what to work on, the evolution of computers, his time in Florence, the challenges of dealing with misinterpretations of his work, the customs and constraints in various fields like venture capital and essay writing, the renaming of Y Combinator, and the impact of rapid change on different industries.</b>

In [18]:
pip install gradio

In [28]:
# Install necessary packages
%pip install -q llama-index llama-index-vector-stores-pinecone gradio

# --- Imports ---
import os
import logging
import sys
import gradio as gr
from IPython.display import Markdown, display

from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# --- Logging ---
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# --- Set API Keys ---
os.environ["PINECONE_API_KEY"] = "pcsk_3hG9JF_HUuVnMGYDVoeTZctGjvTJRASUnQQjnDVFNUMX2ntkdBsfhLbbYxapdLzTaedRaK"


api_key = os.environ["PINECONE_API_KEY"]


# --- Initialize Pinecone ---
pc = Pinecone(api_key=api_key)
index_name = "quickstart"
dimension = 1536

# Delete index if exists (optional)
if index_name in [idx['name'] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load Data ---
!mkdir -p 'data/paul_graham/'
!wget -q 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#change directory
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# --- Query Engine ---
query_engine = index.as_query_engine()

# --- Query Function ---
def ask_question(prompt):
    try:
        response = query_engine.query(prompt)
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"

# --- Gradio App ---
with gr.Blocks(css="""
.gradio-container {
    background-color: #f8f9fa;
    font-family: 'Segoe UI', sans-serif;
}
textarea, input, button {
    font-size: 16px !important;
}
""") as demo:
    gr.Markdown("## 🧠 Paul Graham Essay Q&A with LlamaIndex + Pinecone")
    gr.Markdown("""
Welcome! This app allows you to explore insights from Paul Graham's essay using semantic search powered by **LlamaIndex** and **Pinecone**.
""")

    with gr.Accordion("ℹ️ What is Pinecone Vector Indexing?", open=False):
        gr.Markdown("""
**Pinecone** is a vector database that stores document embeddings (numeric representations of meaning). When you ask a question, it's converted into a vector and compared against stored vectors to find the most relevant answers — even if they don't match word-for-word.
""")

    gr.Markdown("### 📖 Ask your question below:")
    with gr.Row():
        user_input = gr.Textbox(placeholder="E.g., What does Paul Graham say about startups?", label="Your Question", lines=2)
    with gr.Row():
        output = gr.Textbox(label="Answer", lines=6)
    with gr.Row():
        submit_btn = gr.Button("🔍 Search Essay")
        clear_btn = gr.Button("🧹 Clear")
        submit_btn.click(fn=ask_question, inputs=user_input, outputs=output)
       # Clear both input and output
        clear_btn.click(fn=lambda: ("", ""), inputs=None, outputs=[user_input, output])

demo.launch()

Upserted vectors:   0%|          | 0/22 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://22420a44509624d6c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [25]:
# --- Install dependencies ---
# %pip install -q llama-index llama-index-vector-stores-pinecone gradio

import os
import sys
import logging
import gradio as gr
from pinecone import Pinecone, ServerlessSpec
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore

# --- Logging ---
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# --- API Key ---
api_key = os.environ["PINECONE_API_KEY"]

# --- Pinecone Initialization ---
pc = Pinecone(api_key=api_key)
index_name = "paul-graham-index"
dimension = 1536

# Delete index if exists (optional)
if index_name in [idx['name'] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load Essay Document ---
!mkdir -p 'data/paul_graham/'
!wget -q 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'
documents = SimpleDirectoryReader("data/paul_graham/").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pc.Index(index_name))
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine()

# --- Query Function ---
def ask_question(prompt):
    try:
        response = query_engine.query(prompt)
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"

# --- Gradio UI ---
with gr.Blocks(css="""
.gradio-container {
    background-color: #f8f9fa;
    font-family: 'Segoe UI', sans-serif;
}
textarea, input, button {
    font-size: 16px !important;
}
""") as demo:
    gr.Markdown("## 🧠 Paul Graham Essay Q&A with LlamaIndex + Pinecone")
    gr.Markdown("""
Welcome! This app allows you to explore insights from Paul Graham's essay using semantic search powered by **LlamaIndex** and **Pinecone**.
""")

    with gr.Accordion("ℹ️ What is Pinecone Vector Indexing?", open=False):
        gr.Markdown("""
**Pinecone** is a vector database that stores document embeddings (numeric representations of meaning). When you ask a question, it's converted into a vector and compared against stored vectors to find the most relevant answers — even if they don't match word-for-word.
""")

    gr.Markdown("### 📖 Ask your question below:")
    with gr.Row():
        user_input = gr.Textbox(placeholder="E.g., What does Paul Graham say about startups?", label="Your Question", lines=2)
    with gr.Row():
        output = gr.Textbox(label="Answer", lines=6)
    with gr.Row():
        submit_btn = gr.Button("🔍 Search Essay")
        submit_btn.click(fn=ask_question, inputs=user_input, outputs=output)

demo.launch()


ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '2e56fc630b07ceaec777d9c956ade0c8', 'date': 'Wed, 18 Jun 2025 19:44:20 GMT', 'server': 'Google Frontend', 'Content-Length': '198', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Request failed. You've reached the max serverless indexes allowed in project AI Agents (5). To add more serverless indexes, upgrade your plan."},"status":403}
